In [71]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.layers import Layer,Input,InputSpec
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from tensorflow.keras.layers import Conv2D,Flatten,ConvLSTM2D,Conv1D
from tensorflow.keras import backend as K

from tensorflow.keras.backend import argmax
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from tensorflow.keras.utils import to_categorical


In [174]:
def create_dataset(dataset, look_back=1):
    from sklearn.preprocessing import MinMaxScaler
    print('dataset ',dataset) 
    #scaler = MinMaxScaler(feature_range=(0, 0.999))
    #YM = scaler.fit_transform(dataset)
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back)]
        dataY.append(dataset[i + look_back]) 
        dataX.append(a)
    #dataXX= numpy.reshape(dataX, (dataX.shape[0], dataX.shape[1],1))
    
    return numpy.array(dataX), numpy.array(dataY)

In [175]:
class ReformatLayer(Layer):
    def __init__(self, n_window,   **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ReformatLayer, self).__init__(**kwargs)
        self.n_window = n_window
        #self.initial_weights = weights
        #self.input_spec = InputSpec(ndim=2)

 

    def build(self, input_shape):
        print('build', input_shape)
        #self.input_spec = [InputSpec(shape=input_shape)]
        # assert len(input_shape) == 1
        #input_dim = input_shape[1]
        #self.input_spec = InputSpec(dtype=tf.int32, shape=(None, input_dim))
        #self.clusters = self.add_weight('clusters',(self.n_clusters, input_dim), initializer='glorot_uniform')
        self.built = True

    def call(self, inputs, **kwargs):
        print('call',inputs)
        x,y=create_dataset(inputs,self.n_window)
        print('x.shape',x.shape,'  ',y.shape)
        #x1= numpy.reshape(x, (x.shape[0], x.shape[1],1))

        return x

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_window

    def get_config(self):
        config = {'n_window': self.n_window}
        base_config = super(ReformatLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
     

In [176]:
xx = Input(shape=[1,10])
xx.shape

TensorShape([None, 1, 10])

In [177]:
x = tf.ones((10),dtype=tf.int32)
y=[3,4,5,6]
z=tf.concat([y,x,y],0)
#x=[1,1]
#tf.backend.eval(x)
wi=Input([1,2,3])
z

<tf.Tensor: id=4271, shape=(18,), dtype=int32, numpy=array([3, 4, 5, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 4, 5, 6], dtype=int32)>

In [178]:
linear_layer = ReformatLayer( 5)
y = linear_layer(z)
print(y)

build (18,)
call tf.Tensor([3 4 5 6 1 1 1 1 1 1 1 1 1 1 3 4 5 6], shape=(18,), dtype=int32)
dataset  tf.Tensor([3 4 5 6 1 1 1 1 1 1 1 1 1 1 3 4 5 6], shape=(18,), dtype=int32)
x.shape (13, 5)    (13,)
[[3 4 5 6 1]
 [4 5 6 1 1]
 [5 6 1 1 1]
 [6 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 3]
 [1 1 1 3 4]
 [1 1 3 4 5]]


In [111]:
class ComputeSum(Layer):

    def __init__(self):
        super(ComputeSum, self).__init__()
        #self.total = tf.Variable(initial_value=tf.zeros((input_dim,)),
        #                     trainable=False)

    def build(self, input_shape):
        print('build', input_shape)
        self.total = tf.Variable(initial_value=tf.zeros((input_shape[1],)),
                             trainable=False)
    def call(self, inputs):
        self.total.assign_add(tf.reduce_sum(inputs, axis=0))
        return self.total

In [ ]:
x = tf.ones((3, 4))
#x=Input(numpy.array([[1.,1.,2.][1,4,6]]))
y=x[0:2]

In [136]:
x1=[1,2,3]
x2=[3,4,5]
z=tf.concat([x1,x2],0)
print(z)
z2=z[2:4]
z2

tf.Tensor([1 2 3 3 4 5], shape=(6,), dtype=int32)


<tf.Tensor: id=443, shape=(2,), dtype=int32, numpy=array([3, 3], dtype=int32)>

In [125]:
my_sum = ComputeSum()
y = my_sum(x)
print(y.numpy())
y = my_sum(x)
print(y.numpy())

build (3, 4)
[3. 3. 3. 3.]
[6. 6. 6. 6.]
